In [ ]:
import tkinter as tk
from tkinter import messagebox, Toplevel, Label, Button
import cv2
import numpy as np
import json
from keras_facenet import FaceNet
from ultralytics import YOLO
from sklearn.metrics.pairwise import cosine_similarity
from PIL import Image, ImageTk
import imgaug.augmenters as iaa
from datetime import datetime
import os

# Initialize YOLO and FaceNet
yolo_model = YOLO('/Users/klnimri/Downloads/FaceReco.pt')
embedder = FaceNet()

# Paths for saving embeddings and attendance records
embeddings_path = "/Users/klnimri/Desktop/DeepLearningFinalProject/face_embeddings.json"
attendance_path = "/Users/klnimri/Desktop/DeepLearningFinalProject/attendance_records.json"

# Initialize embeddings database and attendance records
database = {}
attendance_records = {}

# Load existing embeddings from file
def load_embeddings():
    global database
    if os.path.exists(embeddings_path) and os.path.getsize(embeddings_path) > 0:
        with open(embeddings_path, "r") as file:
            database = json.load(file)
        for student_id in database:
            database[student_id]['embedding'] = np.array(database[student_id]['embedding'])
    else:
        database = {}

# Save embeddings to file
def save_embeddings():
    save_data = {k: {'name': v['name'], 'embedding': v['embedding'].tolist()} for k, v in database.items()}
    with open(embeddings_path, "w") as file:
        json.dump(save_data, file)

# Load existing attendance records
def load_attendance_records():
    global attendance_records
    if os.path.exists(attendance_path) and os.path.getsize(attendance_path) > 0:
        with open(attendance_path, "r") as file:
            attendance_records = json.load(file)
    else:
        attendance_records = {}

# Save attendance records to file
def save_attendance_records():
    with open(attendance_path, "w") as file:
        json.dump(attendance_records, file, indent=4)

# Define augmentations
augmentation_pipeline = iaa.Sequential([
    iaa.Affine(scale=(0.9, 1.1), rotate=(-10, 10)),
    iaa.Fliplr(0.5),
])

# Capture, augment, and save embeddings
def create_student():
    def capture_and_save():
        ret, frame = cap.read()
        if not ret:
            messagebox.showerror("Error", "Could not access webcam.")
            return

        results = yolo_model(frame)
        if len(results[0].boxes) > 0:
            x1, y1, x2, y2 = map(int, results[0].boxes[0].xyxy[0])
            face = frame[y1:y2, x1:x2]
            face_rgb = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
            embeddings = []

            for _ in range(5):
                augmented_face = augmentation_pipeline(image=face_rgb)
                embedding = embedder.embeddings([augmented_face])[0]
                embeddings.append(embedding)

            avg_embedding = np.mean(embeddings, axis=0)

            name = name_entry.get()
            student_id = id_entry.get()
            if name and student_id:
                database[student_id] = {'name': name, 'embedding': avg_embedding}
                save_embeddings()
                messagebox.showinfo("Success", f"{name} has been enrolled!")
                signup_win.destroy()

    signup_win = Toplevel(root)
    signup_win.title("Sign Up")
    center_window(signup_win, 300, 200)  # Center the Sign Up window

    Label(signup_win, text="Name:").pack()
    name_entry = tk.Entry(signup_win)
    name_entry.pack()
    Label(signup_win, text="Student ID:").pack()
    id_entry = tk.Entry(signup_win)
    id_entry.pack()
    Button(signup_win, text="Capture and Save", command=capture_and_save).pack(pady=10)

# Center window on screen
def center_window(window, width, height):
    screen_width = window.winfo_screenwidth()
    screen_height = window.winfo_screenheight()
    x = (screen_width - width) // 2
    y = (screen_height - height) // 2
    window.geometry(f"{width}x{height}+{x}+{y}")

# Display camera feed with real-time recognition
def update_camera_feed():
    ret, frame = cap.read()
    if ret:
        results = yolo_model(frame)
        if len(results[0].boxes) > 0:
            x1, y1, x2, y2 = map(int, results[0].boxes[0].xyxy[0])
            face = frame[y1:y2, x1:x2]
            face_rgb = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
            embedding = embedder.embeddings([face_rgb])[0]

            best_match = None
            best_similarity = 0
            for student_id, data in database.items():
                similarity = cosine_similarity([embedding], [data['embedding']])[0][0]
                if similarity > best_similarity:
                    best_similarity = similarity
                    best_match = (data['name'], student_id)

            if best_similarity >= 0.8 and best_match:
                name, student_id = best_match
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(frame, f"{name} ({student_id})", (x1, y1 - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        img = Image.fromarray(frame_rgb)
        imgtk = ImageTk.PhotoImage(image=img)
        camera_label.imgtk = imgtk
        camera_label.configure(image=imgtk)

    camera_label.after(20, update_camera_feed)  # Reduce delay here

# Take attendance
def take_attendance():
    ret, frame = cap.read()
    if not ret:
        return

    results = yolo_model(frame)
    if len(results[0].boxes) > 0:
        x1, y1, x2, y2 = map(int, results[0].boxes[0].xyxy[0])
        face = frame[y1:y2, x1:x2]
        face_rgb = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
        embedding = embedder.embeddings([face_rgb])[0]

        best_match = None
        best_similarity = 0
        for student_id, data in database.items():
            similarity = cosine_similarity([embedding], [data['embedding']])[0][0]
            if similarity > best_similarity:
                best_similarity = similarity
                best_match = (data['name'], student_id)

        if best_similarity >= 0.7:
            name, student_id = best_match
            now = datetime.now()
            date_str = now.strftime("%Y-%m-%d")
            time_str = now.strftime("%H:%M:%S")

            if date_str not in attendance_records:
                attendance_records[date_str] = {}

            if student_id not in attendance_records[date_str]:
                attendance_records[date_str][student_id] = {'name': name, 'times': []}
            
            attendance_records[date_str][student_id]['times'].append(time_str)
            save_attendance_records()

            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(frame, f"{name} ({student_id})", (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
            messagebox.showinfo("Attendance", f"Attendance recorded for {name} (ID: {student_id})")

    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    img = Image.fromarray(frame_rgb)
    imgtk = ImageTk.PhotoImage(image=img)
    camera_label.imgtk = imgtk
    camera_label.configure(image=imgtk)

# Show attendance records
def show_records():
    records_win = Toplevel(root)
    records_win.title("Attendance Records")
    center_window(records_win, 400, 400)  # Center the Records window

    for date, students in attendance_records.items():
        date_label = Label(records_win, text=f"{date}")
        date_label.pack()

        for student_id, data in students.items():
            attendance_count = len(data['times'])
            record_label = Label(records_win, text=f"{data['name']} (ID: {student_id}) - Times: {', '.join(data['times'])} - Count: {attendance_count}")
            record_label.pack()

root = tk.Tk()
root.title("Attendance System")
center_window(root, 650, 650)  

cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 320)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 240)

camera_label = tk.Label(root)
camera_label.pack()

tk.Button(root, text="Sign Up", command=create_student).pack(pady=10)
tk.Button(root, text="Take Attendance", command=take_attendance).pack(pady=10)
tk.Button(root, text="Records", command=show_records).pack(pady=10)

load_embeddings()
load_attendance_records()
root.after(0, update_camera_feed)
root.mainloop()

cap.release()


0: 480x640 (no detections), 58.6ms
Speed: 2.3ms preprocess, 58.6ms inference, 1.8ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 sad, 61.1ms
Speed: 1.0ms preprocess, 61.1ms inference, 0.9ms postprocess per image at shape (1, 3, 480, 640)
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step

0: 480x640 1 sad, 54.1ms
Speed: 1.4ms preprocess, 54.1ms inference, 0.6ms postprocess per image at shape (1, 3, 480, 640)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step


2024-11-01 18:02:43.453 python[13935:1137918] +[IMKClient subclass]: chose IMKClient_Legacy
2024-11-01 18:02:43.453 python[13935:1137918] +[IMKInputSession subclass]: chose IMKInputSession_Legacy



0: 480x640 1 sad, 62.6ms
Speed: 1.2ms preprocess, 62.6ms inference, 0.3ms postprocess per image at shape (1, 3, 480, 640)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step

0: 480x640 1 sad, 55.7ms
Speed: 1.0ms preprocess, 55.7ms inference, 0.6ms postprocess per image at shape (1, 3, 480, 640)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step

0: 480x640 1 sad, 63.3ms
Speed: 0.9ms preprocess, 63.3ms inference, 0.3ms postprocess per image at shape (1, 3, 480, 640)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step

0: 480x640 1 sad, 54.2ms
Speed: 1.0ms preprocess, 54.2ms inference, 0.6ms postprocess per image at shape (1, 3, 480, 640)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step

0: 480x640 1 sad, 50.5ms
Speed: 1.1ms preprocess, 50.5ms inference, 0.3ms postprocess per image at shape (1, 3, 480, 640)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step

0: 480x640 1 sad, 51.9ms
Speed: 1.2ms preprocess, 51.9ms inference, 0.5ms postprocess per image at shape (1, 3, 480, 640)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step

0: 480x640 1 sad, 52.6ms
Speed: 0